# Chat Completion API

In [ ]:
%pip install openai

## 환경설정
1. openai 사이트에서 api key 발급
2. google colab - secret탭에 `OPENAI_API_KEY`  등록


In [1]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI

client = OpenAI(api_key = OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "반가워 나는 '한성규'라고 해"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=0.6,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'반가워요, 한성규님! 어떻게 지내고 계신가요? 궁금한 점이나 이야기하고 싶은 것이 있다면 언제든지 말씀해 주세요.'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction  지시사항
2. Context      문맥
3. Input Data/Example   입력/ 예시
4. Output Indicator       출력 지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정

- 프랑스AFP 속보시스템에서 도입되어 사용

In [ ]:
title_before =  '조성렬의 아무말대잔치 미친 텐션의 댄스클럽에 가입해보세요'

# 역할 / 페르소나/ 지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정작업을 수행해주세요

- 기사 제목이 명확하고, 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.


### Steps ###
1. 기사제목을 읽고 주요 내용을 이해하세요.
2. 제목이 전달하고자 하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결하게 정리하세요.

### Output Format ###
기사의 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.
- 원래 제목 : [기사 원래 제목]
- 교정 제목 : [교정 기사 제목]

### Examples ###
- 원래 제목 : ' 어제 서울에서 큰 불이 나 수백명이 대피했다.'
- 교정 제목 : ' 서울 대형 화재, 수백명 대피'


### Extra instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""
user_message = f'''
다음 기사 제목을 간결히 수정해주세요.

{title_before}
'''


response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },

  temperature=0.6,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
response.choices[0].message.content

'- 원래 제목 : 조성렬의 아무말대잔치 미친 텐션의 댄스클럽에 가입해보세요\n- 교정 제목 : 조성렬의 댄스클럽, 새로운 즐거움을 경험해보세요'

In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)


- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길~
- 교정 제목: "주말 폭우 예보, 안전에 유의하세요"


## 영문텍스트에 anki 단어장 만들기

## 연애코치 ReAct

In [3]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로서 사용자의 고민을 해결해주세요


    ### Output Format ###
    1. 상황 분석:

    2. 행동 계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재 상황.

    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [4]:
print(dating_coach('여자친구와 6주년을 앞두고 있는데 어떻게 준비하면 좋을까요?'))

1. 상황 분석:
   - 6주년이라는 중요한 기념일을 맞이하기 위해 특별한 준비가 필요합니다.
   - 오랜 관계인 만큼 서로의 취향과 함께했던 추억들을 고려한 맞춤형 서프라이즈 또는 기념일 계획이 중요합니다.
   - 여자친구가 무엇을 좋아하는지를 잘 파악하여 그에 맞는 선물을 준비하면 더욱 의미 있는 기념일이 될 것입니다.

2. 행동 계획:
   - 먼저 여자친구의 취향, 관심사, 혹은 평소에 이야기했던 꿈이나 소원들을 떠올려 보십시오.
   - 다음으로, 작은 이벤트를 기획해보세요. 예를 들어, 특별한 저녁 식사 예약, 주말 여행, 또는 개인적인 선물(손편지, 사진 앨범 등)을 준비할 수 있습니다.
   - 기념일에 맞춰 특별한 메시지를 담은 편지를 작성하여 그녀에게 감동을 줄 수 있도록 합니다.
   - 리뷰나 데이트 장소들에 대한 추억을 회상하며 그녀와 함께하는 특별한 순간을 만들어보세요.

3. 실행:
   - 기념일이 가까워지면 준비한 장소나 이벤트에 대해 미리 예약을 하고 필요한 물품을 구매합니다.
   - 선물과 편지를 정성스럽게 포장합니다.
   - 기념일 당일, 여자친구에게 깜짝 이벤트를 진행하며, 준비한 내용들을 자연스럽게 이야기하며 회상하는 시간을 가집니다.
   - 이 모든 과정을 통해 서로의 사랑을 더욱 깊이 나누고 의미 있는 시간을 보내세요. 

고민하고 준비하는 이 과정 자체가 사랑을 더욱 깊게 만들어 줄 것입니다. 좋고 특별한 6주년을 만들어 보세요!


## 냉털마스터 ReAct
- 사용자는 냉장고에 남아있는 음식 재료르 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- reasoning / action을 끌어낼 수 있는 적절한 프롬프팅을 작성하세요.

In [10]:
def nengboo(jjam, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    남은 음식 재료를 활용해 최상의 맛을 낼 수 있는 요리의 조리법을 알려주는 냉털마스터입니다.
    웹에서 레시피 검색을 통해 맛있는 요리 하나를 추천해주세요.
    1인분을 만들 것이고, 재료들의 계량도 해야합니다.

    ### Output Format ###
    1. 냉털 메뉴와 재료:

    2. 단계별 조리법:

    3. 추가하면 좋을 재료:

    4. 레시피 출처:
    """

    user_message = f"""
    냉장고에 남은 음식.

    {jjam}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [11]:
print(nengboo('양념치킨, 두부 , 고수'))

1. 냉털 메뉴와 재료:
   - 메뉴: 양념치킨 두부 덮밥
   - 재료:
     - 양념치킨: 100g
     - 두부: 150g
     - 고수: 10g (선택적으로)
     - 밥: 1공기 (약 150g)
     - 간장: 1큰술
     - 참기름: 1작은술
     - 후추: 약간

2. 단계별 조리법:
   1. 두부는 1cm 두께로 썰어주세요.
   2. 팬에 기름을 두르고 중불에서 두부를 앞뒤로 노릇하게 구워줍니다.
   3. 구운 두부에 간장과 참기름을 넣고 약간 졸여 맛을 스며들게 합니다.
   4. 따로 데운 양념치킨을 두부 위에 올립니다.
   5. 준비한 밥을 접시에 담고, 그 위에 두부와 양념치킨을 올립니다.
   6. 마지막으로 고수를 잘게 썰어 위에 뿌리거나 옆에 곁들여 서빙합니다.
   7. 후추를 약간 뿌리면 더욱 맛이 살아납니다.

3. 추가하면 좋을 재료:
   - 쌈장 또는 마늘
   - 치킨 스톡 또는 육수 (보다 풍미 있는 맛을 위해)
   - 고추나 청양고추 (매운맛을 원할 경우)

4. 레시피 출처: 개인 창작 요리법


In [14]:
from IPython.display import Markdown
display(Markdown(nengboo('양장피')))

1. 냉털 메뉴와 재료:
   - 양장피 (1인분)
   - 오이 1/4개 (채 썰기)
   - 당근 1/4개 (채 썰기)
   - 적양배추 1/4컵 (채 썰기)
   - 삶은 새우 3-4마리 (선택 사항)
   - 참기름 1큰술
   - 간장 1큰술
   - 식초 1큰술
   - 고춧가루 1/2작은술 (선택 사항)

2. 단계별 조리법:
   1. 양장피는 미리 준비된 상태라면 그대로 사용합니다.
   2. 오이, 당근, 적양배추를 채 썰어 준비합니다.
   3. 삶은 새우를 사용할 경우, 끓는 물에 새우를 살짝 삶아 준비합니다.
   4. 큰 그릇에 양장피, 썬 야채, 새우를 넣습니다.
   5. 다른 볼에서 참기름, 간장, 식초, 고춧가루를 섞어 드레싱을 만듭니다.
   6. 드레싱을 양장피와 야채에 부어서 잘 섞습니다.
   7. 접시에 담아 서빙합니다.

3. 추가하면 좋을 재료:
   - 숙주나물 (영양가를 더하기 위해)
   - 후리카케 (일본식 양념)
   - 통깨 (토핑용)

4. 레시피 출처: 개인적인 경험 및 다양한 요리 사이트를 참고하여 조합하였습니다.

## 면접질문 생성 JSON 출력


In [18]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}
"""

    user_message = f"""
채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [19]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |

"""
output = job_interview(job_posting)
print(type(output))

{
    "hard_skill": [
        {
            "question": "Python을 사용하여 머신러닝 인프라를 구축한 경험에 대해 설명해 주세요.",
            "answer": "저는 Python을 사용하여 머신러닝 인프라를 구축한 경험이 있습니다. 특히, 데이터를 처리하고 클린징하기 위해 Pandas와 NumPy 라이브러리를 활용했습니다. 모델 훈련과 후처리를 위해 Scikit-learn을 이용해 최적의 파라미터 튜닝을 진행했습니다."
        },
        {
            "question": "Docker와 Kubernetes를 사용하여 CI/CD 파이프라인을 설정한 경험이 있습니까?",
            "answer": "네, 저는 Docker를 이용해 애플리케이션의 컨테이너 이미지를 빌드하고, Kubernetes를 활용해 클러스터에서 이 이미지를 배포한 경험이 있습니다. Jenkins를 사용하여 CI/CD 파이프라인을 설정하고, 코드 변경 사항이 있을 때마다 자동으로 테스트와 배포가 이루어지도록 구성했습니다."
        },
        {
            "question": "모니터링 및 로깅 시스템을 설계 및 구현한 경험에 대해 말씀해 주세요.",
            "answer": "저는 Prometheus와 Grafana를 사용하여 머신러닝 시스템의 성능을 모니터링하는 시스템을 설계했습니다. 각 서비스의 메트릭을 수집하고 시각화하여, 성능 저하를 조기에 탐지할 수 있도록 했습니다. 또한, ELK 스택을 활용하여 로그 수집 및 분석을 효율적으로 수행했습니다."
        }
    ],
    "soft_skill": [
        {
            "question": "AI 모델 개발자와 협업할 때 가장 중요하다고 생각하는 요소는 무엇인가요?",
            "answer": "AI 모델 개발자와의 협업에서 가장 중요한 요소는 원활한

In [ ]:
import json

# 응답데이터가 json 형식이 아니라면 오류 발생!
data = json.load(output)
print(type(data))


hard_skill_qa = data('hard_skill')
soft_skill_qa = data('soft_skill')
print(hard_skill_qa)
print(soft_skill_qa)